<a href="https://colab.research.google.com/github/spacexjedi/data-science-101/blob/master/rp_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Identificar três possíveis eventos em sinais gravados com o OpenBCI. Os eventos são os seguintes:
• Mordida ou aperto de mandíbula (jaw clench);
• Piscar dos olhos (blink);
• Ritmos alpha: onda evocada no estado de meditação, aumentando as frequências que vão de 8 à 13 Hz.
• E ritmos beta: onda evocada no estado de concentração, aumentando as frequências que vão de 13 à 32 Hz.
O arquivo "Gravação_EEG_OpenBCI-GUI.mp4" contém a explicação dos períodos em que são evocados os quatro eventos.
O arquivo "OpenBCI-RAW-2020-11-02_01-35-26.txt" contém os dados gravados pelo vídeo anterior, contendo os quatro eventos.
O arquivo "OpenBCI_GUI-v5-meditation.txt" contém dados gravados no OpenBCI de um longo período de meditação, evocando intensamente os ritmos alpha.
Vocês deverão entregar um código que indique o momento em que um evento inicia e termina, além de nomear o evento ocorrido.
"""

In [ ]:
!pip install mne

     |████████████████████████████████| 6.8MB 5.2MB/s 


In [ ]:
!pip install pylsl

     |████████████████████████████████| 911kB 5.9MB/s 


In [1]:
# é preciso carregar o arquivo OpenBCI-RAW-2020-11-02_01-35-26.txt na pasta antes de rodar, não fica salvo no google colab
# https://github.com/spacexjedi/data-science-101/tree/master/MAMEM_SSVEP/T2

In [ ]:
#bibliotecas
import mne
import matplotlib.pyplot as plt 
from pylsl import*
import numpy as np
import re
import time
import scipy

In [ ]:
#carregar arquivo
filepath = './OpenBCI-RAW-2020-11-02_01-35-26.txt'
def read_file(filepath):
  data = open(filepath).readlines()
  labels = StreamInfo('OpenBCI', 'EEG', 8, 256, 'float32', 'rp')
  stream = StreamOutlet(labels)
  for i in data:
    if re.search(r'^\d', i):
      line = [float(e[1:]) for e in line.split(',')[1:9]]
      stream.push_sample(line)
      sleep(1/256)

In [ ]:
#processamento
freq_dict = {'delta':(0.5,3),'theta':(4,7),'alpha':(8,12),'beta':(12,30),'gamma':(33,90)}
width = 10
chunk = 250

streaming = StreamInlet(resolve_stream('type', 'EEG')[0])
values, timestamp = streaming.pull_chunk(timeout=3, max_samples=chunk)
buffer = np.array(values)

for i in range(width - 2):
  values, timestamp = streaming.pull_chunk(timeout=3, max_samples=chunk)
  buffer = np.concatenate((buffer, values))
  print(i)
  print(buffer.shape)

In [ ]:
while True:
  values, timestamp = streaminig.pull_chunk(timeout=3, max_samples=chunk)
  buffer = np.concatenate((buffer, values))
  freq, psd = welch(buffer, fs=250., nperseg=8)
  output = {band: np.mean(psd[np.where((freq >= lf) & (freq <= hf))]) for band, (lf, hf) in eeg_bands.items()}
  print(output)
  print(buffer.shape)
  buffer = np.delete(buffer, slice(chunk), axis=0)
  print(buffer.shape)

In [ ]:
#matplot
value_alpha = freq_dict['alpha']
value_beta = freq_dict['beta']
value_delta = freq_dict['delta']
value_theta = freq_dict['theta']
value_gamma = freq_dict['gamma']
value = [value_alpha, value_beta, value_delta, value_theta, value_gamma]
names = ['alpha', 'beta', 'delta', 'theta', 'gamma']
plt.bar(names, value, color=['pink', 'blue', 'green', 'purple', 'yellow'])
plt.show()

In [2]:
#Referências
#Documentação: https://docs.python.org/3/
#Livro: Python para Desenvolvedores, Luis Borges
#Vídeo sobre OpenBCI fornecido pelo professor